# Set up

### Import Python packages to use in the script

In [2]:
import skimage as sk
from skimage.io import imread, imshow, imsave
import numpy as np
import os
from glob import glob
import napari
from cellpose import models, io, utils
import pandas as pd
import pyclesperanto as cle
import skan

### Define the functions that will run different steps/tasks on the data

In [3]:
def normalize_images(input_image,tophat_radius):
    input_gpu = cle.push(input_image)
    #normalizing the image stack
    equalized_intensities_stack = cle.create_like(input_gpu)
    a_slice = cle.create([input_gpu.shape[1], input_gpu.shape[2]])
    num_slices = input_gpu.shape[0]
    mean_intensity_stack = cle.mean_of_all_pixels(input_gpu)
    corrected_slice = None
    for z in range(0, num_slices):
        # get a single slice out of the stack
        cle.copy_slice(input_gpu, a_slice, z)
        # measure its intensity
        mean_intensity_slice = cle.mean_of_all_pixels(a_slice)
        # correct the intensity
        correction_factor = mean_intensity_slice/mean_intensity_stack
        corrected_slice = cle.multiply_image_and_scalar(a_slice, corrected_slice, correction_factor)
        # copy slice back in a stack
        cle.copy_slice(corrected_slice, equalized_intensities_stack, z)
    #background subtraction (increase the signal to noise ratio for improved segmentation results)
    background_subtracted_top_hat = cle.top_hat_sphere(equalized_intensities_stack,radius_x=tophat_radius,radius_y=tophat_radius,radius_z=tophat_radius)
    #pull data off gpu
    input_pull = cle.pull(input_gpu)
    background_subtracted_top_hat_pull = cle.pull(background_subtracted_top_hat)
    equalized_intensities_stack_pull = cle.pull(equalized_intensities_stack)
    return background_subtracted_top_hat

In [ ]:
def get_measurements(mask,img,props):
    df = sk.measure.regionprops_table(mask,img,properties=props)
    return df

In [ ]:
def scaled_vol(df,voxel):
    scaled_vol = []
    for a in np.asarray(df['area']).astype(int):
        scaled_vol.append(a*voxel)
    vol_df = pd.Series(scaled_vol,name='Volume (um^3)')
    scaled_df = pd.concat([df,vol_df],axis=1)
    return scaled_df

In [ ]:
def get_3D_surfacearea(mask_img,scaled_df):
    array = []
    for i in np.asarray(scaled_df['label']).astype(int):
        obj = mask_img == i
        verts, faces, _ , _ = sk.measure.marching_cubes(obj, level=0.0)
        surf_area = sk.measure.mesh_surface_area(verts, faces)
        array.append(surf_area)
    surface_areas = pd.Series(array,name='Surface_Area (um^2)')
    merged_df = pd.concat([scaled_df,surface_areas], axis=1)
    return merged_df

In [ ]:
def save(save_path, img_name, masks, flows, merged_df):
    # make directories if they do not exist
    try:
        masks_path = os.mkdir(os.path.join(save_path,'masks'))
        flows_path = os.mkdir(os.path.join(save_path,'flows'))
        dataframe_path = os.mkdir(os.path.join(save_path,'measurements'))
    # use the expected paths if the directories exist already
    except:
        masks_path = os.path.join(save_path,'masks')
        flows_path = os.path.join(save_path,'flows')
        dataframe_path = os.path.join(save_path,'measurements')
    imsave(os.path.join(masks_path,'masks_'+img_name[:-4]+'.tif'),mask_img)
    imsave(os.path.join(selected_spots_path,'flows_'+img_name[:-4]+'.tif'),selected_spots)
    merged_df.to_csv(os.path.join(dataframe_path,'measurements_'+img_name[:-4]+'.csv'))

# Load in Data

## Two options for loading in data:

### Option 1: Specify the locations of all image containing folders:
Then merge into a long list of images and file names

In [ ]:
CMO_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/Control_MO/*.tif'))
GMO_1ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_1ng/*.tif'))
GMO_2ng_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/GNAS_MO_2ng/*.tif'))

In [ ]:
CMO_imgs = list(map(sk.io.imread,CMO_files))
GMO_1ng_imgs = list(map(sk.io.imread,GMO_1ng_files))
GMO_2ng_imgs = list(map(sk.io.imread,GMO_2ng_files))

In [ ]:
all_files = CMO_files+GMO_1ng_files+GMO_2ng_files # need this for getting image names later
all_img = CMO_imgs+GMO_1ng_imgs+GMO_2ng_imgs

### Option 2: Use a recursive search option to find all images in the parent directory:

In [4]:
all_files = sorted(glob('E:/ParadaKusz_Lab/TIF_Files/**/*.tif',recursive=True)) #finds all tiff images in the subdirectories of TIF_Files
all_img = list(map(sk.io.imread,all_files))

# Run Pipeline

### Optional use of random integer generator if you want to test on single random images first

In [5]:
import random
nums = range(len(all_img))
i = random.randint(min(nums),max(nums))

In [ ]:
tophat_radius = 10
img = normalize_images(all_img[i],tophat_radius) #change the tophat_radius to increase or decrease the background as needed

In [ ]:
io.logger_setup()
model = models.Cellpose(gpu=True,model_type='cyto3') # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _, _ = model.eval(img, diameter=70, channels=channels, z_axis=0,stitch_threshold=0.2,flow_threshold=0.5,cellprob_threshold=1.0)

In [ ]:
#View results of segmentation in Napari
viewer = napari.view_image(all_img[i], name='original img', scale = (4.55,0.3,0.3))
viewer.add_image(img,name='nomarlized img', scale = (4.55,0.3,0.3))
viewer.add_image(masks,name='masks', scale = (4.55,0.3,0.3))

### Run pipeline on all images